# Quickstart: FDFI in 5 Minutes

This tutorial introduces the basics of FDFI (Flow-Disentangled Feature Importance). By the end, you'll be able to:

1. Create an explainer for any model
2. Compute feature importance
3. Interpret the results
4. Get confidence intervals

## Setup

First, let's import the necessary libraries:

In [1]:
import numpy as np
from fdfi.explainers import OTExplainer

# Set random seed for reproducibility
np.random.seed(42)

## Create a Simple Model

Let's create a simple model where we know the true feature importance. Features 0 and 1 are important, the rest are noise:

In [2]:
def model(X):
    """Simple model: y = x0 + 2*x1 + 0.5*x2"""
    return X[:, 0] + 2 * X[:, 1] + 0.5 * X[:, 2]

# Create training data (used as background distribution)
n_samples = 200
n_features = 10
X_train = np.random.randn(n_samples, n_features)

# Create test data to explain
X_test = np.random.randn(20, n_features)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Model predictions for test data: {model(X_test)[:5]}")

Training data shape: (200, 10)
Test data shape: (20, 10)
Model predictions for test data: [-1.36042558 -3.5175806   1.2950158  -0.90179092 -1.77532221]


## Create an Explainer

The `OTExplainer` uses Gaussian optimal transport to compute feature importance:

In [3]:
# Create the explainer
explainer = OTExplainer(
    model,              # The model to explain
    data=X_train,       # Background data
    nsamples=50,        # Monte Carlo samples per feature
)

print("Explainer created!")

Explainer created!


## Compute Feature Importance

Call the explainer on test data to get feature importance:

In [4]:
# Compute feature importance
results = explainer(X_test)

# Print the results
print("Feature Importance (phi_X):")
for i, phi in enumerate(results["phi_X"]):
    print(f"  Feature {i}: {phi:.4f}")

Feature Importance (phi_X):
  Feature 0: 0.7946
  Feature 1: 1.9705
  Feature 2: 0.1916
  Feature 3: 0.0088
  Feature 4: 0.0178
  Feature 5: 0.0071
  Feature 6: 0.0134
  Feature 7: 0.0029
  Feature 8: 0.0006
  Feature 9: 0.0306


## Interpret the Results

The `results` dictionary contains:
- `phi_X`: Feature importance in the original X-space
- `phi_Z`: Feature importance in the disentangled Z-space
- `se_X`, `se_Z`: Standard errors for uncertainty quantification

Higher values indicate more important features. Since our model uses `x0 + 2*x1 + 0.5*x2`, we expect Features 0, 1, and 2 to have the highest importance.

In [5]:
# Sort features by importance
importance = results["phi_X"]
sorted_idx = np.argsort(importance)[::-1]

print("Features ranked by importance:")
for rank, idx in enumerate(sorted_idx):
    print(f"  Rank {rank+1}: Feature {idx} (importance = {importance[idx]:.4f})")

Features ranked by importance:
  Rank 1: Feature 1 (importance = 1.9705)
  Rank 2: Feature 0 (importance = 0.7946)
  Rank 3: Feature 2 (importance = 0.1916)
  Rank 4: Feature 9 (importance = 0.0306)
  Rank 5: Feature 4 (importance = 0.0178)
  Rank 6: Feature 6 (importance = 0.0134)
  Rank 7: Feature 3 (importance = 0.0088)
  Rank 8: Feature 5 (importance = 0.0071)
  Rank 9: Feature 7 (importance = 0.0029)
  Rank 10: Feature 8 (importance = 0.0006)


## Get Confidence Intervals

FDFI provides statistical inference via `conf_int()`:

In [6]:
# Compute confidence intervals
ci = explainer.conf_int(
    alpha=0.05,           # 95% confidence level
    target="X",           # Use X-space importance
    alternative="greater" # Test if importance > 0
)

print("\nConfidence Intervals (95%, one-sided):")
print("-" * 60)
print(f"{'Feature':>8} {'Estimate':>10} {'SE':>10} {'CI Lower':>10} {'P-value':>10}")
print("-" * 60)
for i in range(n_features):
    sig = "*" if ci["reject_null"][i] else ""
    print(f"{i:>8} {ci['phi_hat'][i]:>10.4f} {ci['se'][i]:>10.4f} "
          f"{ci['ci_lower'][i]:>10.4f} {ci['pvalue'][i]:>10.4f} {sig}")

print("\n* = significant at alpha=0.05")


Confidence Intervals (95%, one-sided):
------------------------------------------------------------
 Feature   Estimate         SE   CI Lower    P-value
------------------------------------------------------------
       0     0.7946     0.2767     0.3394     0.0020 *
       1     1.9705     0.6043     0.9765     0.0006 *
       2     0.1916     0.0724     0.0726     0.0041 *
       3     0.0088     0.0225    -0.0282     0.3475 
       4     0.0178     0.0227    -0.0196     0.2169 
       5     0.0071     0.0224    -0.0298     0.3753 
       6     0.0134     0.0225    -0.0237     0.2759 
       7     0.0029     0.0224    -0.0339     0.4482 
       8     0.0006     0.0224    -0.0362     0.4889 
       9     0.0306     0.0235    -0.0080     0.0963 

* = significant at alpha=0.05


## View Summary

Use the built-in `summary()` method for a formatted output:

In [7]:
# Print formatted summary
explainer.summary(alpha=0.05, alternative="greater")

Feature Importance Results
Method: OTExplainer
Number of features: 10
Significance level: 0.05
Alternative: greater
------------------------------------------------------------------------------
 Feature   Estimate    Std Err   CI Lower   CI Upper    P-value   Sig
------------------------------------------------------------------------------
       0     0.7946     0.2767     0.3394        inf     0.0020   ***
       1     1.9705     0.6043     0.9765        inf     0.0006   ***
       2     0.1916     0.0724     0.0726        inf     0.0041   ***
       3     0.0088     0.0225    -0.0282        inf     0.3475      
       4     0.0178     0.0227    -0.0196        inf     0.2169      
       5     0.0071     0.0224    -0.0298        inf     0.3753      
       6     0.0134     0.0225    -0.0237        inf     0.2759      
       7     0.0029     0.0224    -0.0339        inf     0.4482      
       8     0.0006     0.0224    -0.0362        inf     0.4889      
       9     0.0306     0.

"==============================================================================\nFeature Importance Results\n==============================================================================\nMethod: OTExplainer\nNumber of features: 10\nSignificance level: 0.05\nAlternative: greater\n------------------------------------------------------------------------------\n Feature   Estimate    Std Err   CI Lower   CI Upper    P-value   Sig\n------------------------------------------------------------------------------\n       0     0.7946     0.2767     0.3394        inf     0.0020   ***\n       1     1.9705     0.6043     0.9765        inf     0.0006   ***\n       2     0.1916     0.0724     0.0726        inf     0.0041   ***\n       3     0.0088     0.0225    -0.0282        inf     0.3475      \n       4     0.0178     0.0227    -0.0196        inf     0.2169      \n       5     0.0071     0.0224    -0.0298        inf     0.3753      \n       6     0.0134     0.0225    -0.0237        inf     0.27

## Next Steps

Now that you've learned the basics, check out these tutorials:

- **OT Explainer Deep Dive**: Learn more about the Gaussian OT method
- **EOT Explainer**: Entropic OT for non-Gaussian data
- **Confidence Intervals**: Advanced statistical inference